In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl 
import os
import glob
import datetime
import openpyxl
from openpyxl.styles import colors, Font, Color, Border, Side, Alignment, PatternFill
%matplotlib inline

In [2]:
INX_NAME_WORK_LOAD = 4
INX_NAME_DETAIL_COST = 1
SH_COMPANY = ['上海']
GD_COMPANY = ['广上', '广州', '广深']

In [3]:
# work_load_file = r'.\工时比例.xlsx'
# work_load_sheet = '8月'
# detail_cost_file = r'.\费用明细.xlsx'
# detail_cost_sheet = '工资'
# detail_cost_sheet = '公积金'
work_load_file = r'.\2020人员工时分摊_12月.xlsx'
work_load_sheet = '2020预算'
detail_cost_file = r'.\工时分摊表资料模板.xlsx'
detail_cost_sheet = '2020年终预算估'

xls_file_name = '%s汇总_%s.xlsx' % (detail_cost_sheet, datetime.datetime.now().date().strftime('%y%m%d'))
xlsx=pd.ExcelWriter(xls_file_name)

In [4]:
df_work_load = pd.read_excel(work_load_file, sheet_name=work_load_sheet,  header=[0, 1, 2])
df_detail_cost = pd.read_excel(detail_cost_file, sheet_name=detail_cost_sheet)

In [5]:
def clean_work_load(df_work_load):
    valid_columns = df_work_load.columns.values[INX_NAME_WORK_LOAD-1:-2]
    df = df_work_load[valid_columns]
    df = df.set_index(valid_columns[1])
    df = df.fillna(0)
    index_name=df.index.name[0]
    df.index.name=index_name
    return df

def clean_detail_cost(df_detail_cost):
    # valid_columns = df_detail_cost.columns.values[INX_NAME_DETAIL_COST:-1]
    valid_columns = df_detail_cost.columns.values[INX_NAME_DETAIL_COST:]
    df = df_detail_cost[valid_columns]
    df = df.set_index(valid_columns[0])
    df = df.fillna(0)
    return df

def check_data(df_work_load, df_detail_cost):
    df = df_work_load.copy()

    df=df_work_load.copy()
    col_attr=df.columns[0]
    s=(df[col_attr]=='研发') | (df[col_attr]=='非研发')
    if s.all()==False:
        txt = '<工时比例>：属性只能为“研发” “非研发”'
        raise Exception(txt)

    df1 = df.iloc[:, 1:]
    df1 = df1.select_dtypes(exclude=['float64', 'int64'])
    if not df1.empty:
        txt = df1.columns.values
        raise Exception('<工时比例>%s: 有非法值(not float64/int64)' % txt)
        # return '<工时比例>%s: 有非法值(not float64)' % txt

    df1 = df_work_load.sum(axis=1)
    criteria = (df1 < 0.9999) | (df1 > 1.00001)
    if criteria.any():
        txt = df1[criteria].index.values
        raise Exception('<工时比例>%s: 合计不是100%%' % txt)

    col_attr = df.columns[0]
    df1 = df[df[col_attr] != '研发'].iloc[:, -1:]
    criteria = (df1 < 0.9999) | (df1 > 1.00001)
    s_criteria = criteria.T.iloc[0]
    if s_criteria.any():
        txt = df1[s_criteria].index.values
        raise Exception('<工时比例>%s: 非研发类“其他”不为100%%' % txt)

    col_attr = df.columns[0]
    df1 = df[df[col_attr] == '研发'].iloc[:, -1:]
    criteria = (df1 < -0.00001) | (df1 > 0.00001)
    s_criteria = criteria.T.iloc[0]
    if s_criteria.any():
        txt = df1[s_criteria].index.values
        raise Exception('<工时比例>%s: 研发类“其他”不为0%%' % txt)

    df = df_detail_cost.copy()
    s1 = df.index.names!=['姓名']
    s2 = df.columns.to_list()[0:2]!=['部门', '支付归口']
    if s1 or s2:
        txt = '<费用明细>：列标题应该为“姓名” “部门” “支付归口”'
        raise Exception(txt)

    df1 = df.iloc[:, 2:]
    df1 = df1.select_dtypes(exclude=['float64', 'int64'])
    if not df1.empty:
        txt = df1.columns.values
        raise Exception('<费用明细>%s: 有非法值(not float64/int64)' % txt)

    person_in_cost = set(df_detail_cost.index.values)
    person_in_load = set(df_work_load.index.values)
    person_missed = person_in_load-(person_in_cost & person_in_load)
    if len(person_missed) > 0:
        txt = '<工时比例>人员未出现<费用明细>中: %s' % person_missed
        raise Exception(txt)

    person_missed = person_in_cost-(person_in_cost & person_in_load)
    if len(person_missed) > 0:
        txt = '<费用明细>人员未出现<工时比例>中: %s' % person_missed
        raise Exception(txt)

    if len(person_in_load) != len(df_work_load.index.values):
        txt = '<工时比例>中有重复的员工名'
        raise Exception(txt)

    if len(person_in_cost) != len(df_detail_cost.index.values):
        txt = '<费用明细>中有重复的员工名'
        raise Exception(txt)

In [6]:
def flatten_work_load_rd(df_work_load_rd, df_detail_cost):
    df=df_work_load_rd.iloc[:,:-1]
    sr=df.stack([0, 1, 2])
    index_v=sr.index.values
    val=sr.to_list()
    new_index=[]
    for idx in index_v:
        dept=df_detail_cost.loc[idx[0],'部门']
        loc=df_detail_cost.loc[idx[0],'支付归口']
        new_idx=(idx[0], dept, loc, idx[1], idx[2],idx[3])
        new_index.append(new_idx)

    index = pd.MultiIndex.from_tuples(new_index)
    sr_new=pd.Series(val, index=index)
    sr_new.index.names=['人员','部门','属地','大类','项目','项目归属']
    return sr_new

def flatten_work_load_rd1(df_work_load_rd, df_detail_cost):
    df=df_work_load_rd.iloc[:,:-1]
    sr=df.stack([0, 1, 2])
    index_v=sr.index.values
    val=sr.to_list()
    columns=['人员','部门','属地','大类','项目','项目归属','占比']
    df_new=pd.DataFrame(columns=columns)
    for i in range(len(index_v)):
        idx=index_v[i]
        dept=df_detail_cost.loc[idx[0],'部门']
        loc=df_detail_cost.loc[idx[0],'支付归口']
        item=[idx[0], dept, loc, idx[1], idx[2],idx[3], val[i]]
        df_new.loc[i]=item
    df=df_new.set_index('人员')
    return df

def nrd_entire(df_work_load_nrd, df_detail_cost):
    nrd_name=df_work_load_nrd.index.values
    df=df_detail_cost.loc[nrd_name]
    df['大类']='其他'
    df['项目']='0000'
    df['项目归属']=''
    df=df.reset_index()
    df=df.set_index(['姓名','部门','支付归口','大类','项目','项目归属'])
    df.index.names=['人员','部门','属地','大类','项目','项目归属']
    return df

def rd_entire_before_adj(df_work_load_rd, df_detail_cost):
    sr=flatten_work_load_rd(df_work_load_rd, df_detail_cost)
    df_cost=df_detail_cost.iloc[:,2:]
    df = df_cost.mul(sr,axis='index',level=0)
    return df
    # df=df.groupby(['银行支付归口']).sum()
    # sheet_name='%s-研发-全-调整前' % work_load_sheet
    # df.to_excel(xlsx, sheet_name=sheet_name, merge_cells=False) 

def rd_entire_after_adj(df_rd_bf_adj):
    df=df_rd_bf_adj
    sum_all=df.sum().sum()
    sum_sh_emp=df.groupby(['属地']).sum().loc[SH_COMPANY].sum().sum()
    sum_gd_emp=df.groupby(['属地']).sum().loc[GD_COMPANY].sum().sum()
    sum_sh_prj=df.groupby(['项目归属']).sum().loc['上海'].sum().sum()
    sum_gd_prj=df.groupby(['项目归属']).sum().loc['广东'].sum().sum()

    ratio_sh=sum_sh_emp/sum_sh_prj
    ratio_gd=sum_gd_emp/sum_gd_prj
    df_sh_prj=df.loc[(slice(None), slice(None), slice(None), slice(None),slice(None),['上海']),:].mul(ratio_sh)
    df_gd_prj=df.loc[(slice(None), slice(None), slice(None), slice(None),slice(None),['广东']),:].mul(ratio_gd)
    df_merge=pd.concat([df_sh_prj,df_gd_prj])
    return df_merge

In [7]:
def format_xlsx(workbook, sheet_name):
    ws = workbook[sheet_name]

    font = Font(name='Arial', size=10, color=colors.BLACK)
    hd_font = Font(name='Arial', size=10, color=colors.BLACK, bold=True)
    thin = Side(border_style="thin")
    border = Border(top=thin, left=thin, right=thin, bottom=thin)
    alignment = Alignment(horizontal="center", vertical="center")
    fill = PatternFill(fill_type="solid", fgColor="C1CDC1")

    nrows = ws.max_row
    ncols = ws.max_column

    for i in range(nrows):
        for j in range(ncols):
            cell=ws.cell(i+1, j+1)
            cell.number_format = '0.00'
            cell.border = border
            cell.alignment = alignment
            if(cell.font.b):
                cell.font = hd_font
                cell.fill = fill
            else:
                cell.font = font
   

def write_xlsx(df, xlsx, title, month):
    sheet_name = '%s-%s' % (month, title)
    df.to_excel(xlsx, sheet_name=sheet_name, merge_cells=False)


# def write_rd_entire_before_adj(df_work_load_nrd, df_detail_cost, xlsx,month):
#     sr=flatten_work_load_rd(df_work_load_rd, df_detail_cost)
#     df_cost=df_detail_cost.iloc[:,2:]
#     df = df_cost.mul(sr,axis='index',level=0)
#     # df=df.groupby(['银行支付归口']).sum()
#     sheet_name='%s-研发-全-调整前' % work_load_sheet
#     df.to_excel(xlsx, sheet_name=sheet_name, merge_cells=False) 


In [8]:
df_work_load = clean_work_load(df_work_load)
df_detail_cost = clean_detail_cost(df_detail_cost)

In [9]:
check_data(df_work_load, df_detail_cost)

Exception: <费用明细>：列标题应该为“姓名” “部门” “支付归口”

In [10]:
df=df_work_load
col_attr=df.columns[0]
df_work_load_rd=df[df[col_attr]=='研发'].drop([col_attr], axis=1)
df_work_load_nrd=df[df[col_attr]!='研发'].drop([col_attr], axis=1)
# df_work_load_rd

In [11]:
df_nrd=nrd_entire(df_work_load_nrd, df_detail_cost)
df_rd_bf_adj=rd_entire_before_adj(df_work_load_rd, df_detail_cost)
df_rd_af_adj=rd_entire_after_adj(df_rd_bf_adj)

xlsx = pd.ExcelWriter(xls_file_name)

# title = '非研发-全'
# write_xlsx(df_nrd, xlsx, title, month=work_load_sheet)
# df = df_nrd.groupby(['部门']).sum()
# title = '非研发-部门'
# write_xlsx(df, xlsx, title, month=work_load_sheet)
# df = df_nrd.groupby(['支付归口']).sum()
# title = '非研发-属地'
# write_xlsx(df, xlsx, title, month=work_load_sheet)

# title = '研发-全-调整前'
# write_xlsx(df_rd_bf_adj, xlsx, title, month=work_load_sheet)

# title = '研发-全-调整后'
# write_xlsx(df_rd_af_adj, xlsx, title, month=work_load_sheet)

# title = '研发-部门'
# df = df_rd_af_adj.groupby(['部门']).sum()
# write_xlsx(df, xlsx, title, month=work_load_sheet)

# title = '研发-属地'
# df = df_rd_af_adj.groupby(['属地']).sum()
# write_xlsx(df, xlsx, title, month=work_load_sheet)

# title = '研发-大类'
# df = df_rd_af_adj.groupby(['大类']).sum()
# write_xlsx(df, xlsx, title, month=work_load_sheet)

# title = '研发-项目'
# df = df_rd_af_adj.groupby(['项目', '项目归属']).sum()
# write_xlsx(df, xlsx, title, month=work_load_sheet)

# title = '研发-项目归属'
# df = df_rd_af_adj.groupby(['项目归属']).sum()
# write_xlsx(df, xlsx, title, month=work_load_sheet)

# title = '研发-调整前'
# df = df_rd_bf_adj.groupby(['部门','属地','大类','项目','项目归属']).sum()
# write_xlsx(df, xlsx, title, month=work_load_sheet)

# title = '研发-调整后'
# df = df_rd_af_adj.groupby(['部门','属地','大类','项目','项目归属']).sum()
# write_xlsx(df, xlsx, title, month=work_load_sheet)

df1=df_rd_bf_adj
df2=df_nrd
df=pd.concat([df1, df2])

title = '合并-调整前'
df = df.groupby(['部门','属地','大类','项目','项目归属']).sum()
write_xlsx(df, xlsx, title, month=work_load_sheet)


df1=df_rd_af_adj
df2=df_nrd
df=pd.concat([df1, df2])

title = '合并-调整后'
df = df.groupby(['部门','属地','大类','项目','项目归属']).sum()
write_xlsx(df, xlsx, title, month=work_load_sheet)

xlsx.close()

In [12]:
df = df_detail_cost.copy()
df.index.names==['姓名']
df.columns.to_list()[0:2]==['部门', '支付归口']

True